# AWS Clean Up Script

This script will clean up all the resources on an individual/organization's AWS account. This script is still in development and a lot of work is still pending. Although the notebook is self explanatory, a little study on core AWS SDK features will be of use.

### Things to Improve on:

1. Use sets wherever possible to reduce to the number of filter operations
2. Exception handling still needs to be done
3. Alert generation through SNS, MailGun and Twilio needs to be implemented

## Libraries

In [1]:
import boto3, json
from pprint import pprint

## Credentials

In [2]:
def getAWSCreds():
    credentials = json.load(open('../esrcd.json'))
    awsCreds = credentials['aws']
    return(awsCreds)

## Sessions and Resources

In [3]:
def createSession():
    session = boto3.session.Session(aws_access_key_id=getAWSCreds()['accessKey'], 
                                    aws_secret_access_key=getAWSCreds()['secretKey'])
    return(session)

def createResourceObj(resourceName, regionName):
    resource = boto3.resource(resourceName, region_name=regionName, 
                              aws_access_key_id=getAWSCreds()['accessKey'], 
                              aws_secret_access_key=getAWSCreds()['secretKey'])
    return(resource)

def createClient(resourceName,regionName):
    client = boto3.client(resourceName, region_name=regionName,
                          aws_access_key_id=getAWSCreds()['accessKey'], 
                          aws_secret_access_key=getAWSCreds()['secretKey'])
    return(client)

## Regions and Services

In [4]:
def getAllRegions(session):
    return(session.get_available_regions('ec2', partition_name='aws'))

def getAllServices(session):
    return(session.get_available_services())

In [5]:
def getAllServicesAndRegions(session):
    allServices = getAllServices(session)
    serviceDict = {}
    for service in allServices:
        availableRegions = session.get_available_regions(service, partition_name='aws')
        serviceDict[service] = availableRegions
    return(serviceDict)

In [6]:
def getIncludedRegions(excludedRegions, session):
    allRegions = getAllRegions(session)
    return([region for region in allRegions if region not in excludedRegions])

def getIncludedServices(excludedServices, session):
    allServices = getAllServices(session)
    return([service for service in allServices if service not in excludedServices])

def getShortlistedRegions(includedRegions, validRegions):
    return([region for region in includedRegions if region in validRegions])

def getAllRegionsForService(serviceName, session):
    return(session.get_available_regions(serviceName, partition_name='aws'))

# EC2

## Volumes

In [7]:
def getVolumeIds(ec2):
    volumesIds = []
    for vol in ec2.volumes.filter(Filters=[
        {
            'Name': 'attachment.status',
            'Values': [
                'detached'
            ]
        },
    ]):
        volumesIds.append(vol.id)
    return(volumesIds)

In [8]:
def deleteVolumes(includedRegions):
    for region in includedRegions:
        ec2 = createResourceObj('ec2',region)
        volumeIds = getVolumeIds(ec2)
        for volumeId in volumeIds:
            volume = ec2.Volume(volumeId).delete(DryRun=False)
    print("Deleted all detached volumes!")

## Elastic IP

In [9]:
def getIds(ec2):
    elasticIps = []
    for address in ec2.describe_addresses()['Addresses']:
        elasticIps.append({
            "AssociationId" : address['AssociationId'],
            "AllocationId"  : address['AllocationId']
        })
    return(elasticIps)

In [10]:
def disassociateIPs(ec2, elasticIps):
    for ip in elasticIps:
        disassociateRes = ec2.disassociate_address(AssociationId=ip['AssociationId'])
        if(disassociateRes['ResponseMetadata']['HTTPStatusCode'] != 200):
            print("Disassociation failed!")
            pprint(disassociateRes)
        else:
            print("Disassociation response:")
            pprint(disassociateRes)
    print("Disassociate IPs: All done!")    

In [11]:
def releaseIPs(ec2, elasticIps):
    for ip in elasticIps:
        releaseResponse = ec2.release_address(AllocationId=ip['AllocationId'])
        if(releaseResponse['ResponseMetadata']['HTTPStatusCode'] != 200):
            print("Release failed!")
            pprint(releaseResponse)
        else:
            print("Release response:")
            pprint(releaseResponse)
    print("Release IPs: All done!")

In [12]:
def deleteElasticIPs(includedRegions):
    print("Deleting IPs")
    elasticIps = []
    for region in includedRegions:
        print("Region: {}".format(region))
        ec2 = createClient('ec2', region)
        elasticIps = getIds(ec2)
        disassociateIPs(ec2, elasticIps)
        releaseIPs(ec2, elasticIps)
    print("Elastic IPs: All done!")

## EC2 Instances

In [13]:
def deleteEC2Instances(includedRegions):
    instanceIds = listEC2InstanceIds(includedRegions)
    for region in instanceIds.keys():
        ec2 = createResourceObj('ec2', region)
        for instanceId in instanceIds[region]:
            instance = ec2.Instance(id=instanceId).terminate(DryRun=False)
            print("Deleted!")
    print("EC2: Instances: All done!")

def listEC2InstanceIds(includedRegions):
    instanceIds = {}
    for region in includedRegions:
        ec2 = createResourceObj('ec2', region)
        instances = ec2.instances.filter(
        Filters=[{'Name': 'instance-state-name', 'Values': ['running','stopped']}])
        for instance in instances:
            if(region not in instanceIds.keys()):
                instanceIds.setdefault(region,[]).append(instance.id)
            else:
                instanceIds[region].append(instance.id)
    pprint(instanceIds)
    return(instanceIds)

## SNS

In [14]:
def getExcludedTopics():
    topicList = ''
    return(topicList.split())

def getTopics(includedRegions, session, excludedTopics):
    snsRegions = getShortlistedRegions(includedRegions, getAllRegionsForService('sns', session))
    topics = []
    for region in snsRegions:
            print("Region: {}".format(region))
            sns = createResourceObj('sns', region)
            for topic in sns.topics.all():
                if(topic.arn not in excludedTopics):
                    topics.append(topic)
    return(topics)

In [15]:
def deleteTopics(includedRegions, session):
    topics = getTopics(includedRegions, session, getExcludedTopics())
    for topic in topics:
        response = topic.delete()
        if(response['ResponseMetadata']['HTTPStatusCode'] != 200):
            print("Something went wrong")
            #Add Twilio code here
            pprint(response)
        else:
            print("Deleted:")
            print("++++++++++")
            pprint(response)
    print("SNS: Topics: All done!")

## S3 Buckets

In [16]:
def getExcludedBucketNames():
    return('chiragbilling chiragwebsite'.split())

def listValidBuckets(s3):
    validBuckets = []
    for bucket in s3.buckets.all():
        print(bucket.name)
        if(bucket.name not in getExcludedBucketNames()):
            print("Added bucket")
            validBuckets.append(bucket)
    return(validBuckets)

def deleteObjects(s3, validBuckets):
    Delete = {'Objects':[]}
    for bucket in validBuckets:
        for page in bucket.objects.pages():
            for obj in page:
                print(obj.key)
                Delete['Objects'].append({
                    'Key' : obj.key
                })
            bucket.delete_objects(Delete=Delete)
            print("Page complete. Deleted objects")
        print("All pages complete.")

def deleteS3Buckets():
    s3 = createResourceObj('s3', 'ap-south-1')
    validBuckets = listValidBuckets(s3)
    deleteObjects(s3, validBuckets)
    for bucket in validBuckets:
        bucket.delete()
        print("Bucket deleted!")
    print("S3: All done!")

## DynamoDB

In [18]:
def getTableList(dynamo):
    tableList = []
    for table in dynamo.tables.all():
        tableList.append(table)
    return(tableList)

def deleteTableList(tableList):
    for table in tableList:
            response = table.delete()
            if(response['ResponseMetadata']['HTTPStatusCode']!=200):
                print("Something went wrong!")
                #Add twilio code here
            else:
                print("Success!")
                pprint(response)

def deleteDynamoTables(includedRegions, session):
    validRegions = getShortlistedRegions(includedRegions, getAllRegionsForService('dynamodb', session))
    for region in validRegions:
        print("Region: {}".format(region))
        dynamo = createResourceObj('dynamodb', region)
        tableList = getTableList(dynamo)
        deleteTableList(tableList)
    print("DynamoDB: All done!")

## SQS

In [19]:
def getAllQueues(sqs):
    return(sqs.queues.all())

def deleteQueue(queue):
    response = queue.delete()
    if(response['ResponseMetadata']['HTTPStatusCode']!=200):
        print("Something went wrong!")
        pprint(response)
        #Add twilio code here
    else:
        print("Success!")
        pprint(response)

def deleteSQS(includedRegions):
    validRegions = getShortlistedRegions(includedRegions, getAllRegionsForService('sqs', session))
    for region in validRegions:
        sqs = createResourceObj('sqs', region)
        for q in getAllQueues(sqs):
            deleteQueue(q)
    print("SQS: All done!")

# Not Tested:

************************************************************************************************************

## Testing Section

In [10]:
excludedRegions = ['us-east-1']
excludedServices = ['cloudwatch', 'budgets']

In [11]:
session = createSession()
# allRegions = getAllRegions(session)
# allServicesRegions = getAllServicesAndRegions(session)
# allServices = getAllServices(session)
includedRegions = getIncludedRegions(excludedRegions, session)
includedServices = getIncludedServices(excludedServices, session)
# ec2Regions = getAllRegionsForService('ec2', session)
# ec2 = createResourceObj('ec2', 'ap-south-1')
# ec22 = createResourceObj('ec2', 'sa-east-1')

In [72]:
resourceObjCompatibleServices = "cloudformation cloudwatch dynamodb ec2 glacier iam opsworks s3 sns sqs".split()
resourceObjCompatibleServices

['cloudformation',
 'cloudwatch',
 'dynamodb',
 'ec2',
 'glacier',
 'iam',
 'opsworks',
 's3',
 'sns',
 'sqs']

## Changes:

1. use Sets